<a href="https://colab.research.google.com/github/Alexa6748/CV_labs/blob/main/CV_%D0%BB%D0%B0%D0%B11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy
import numpy as np
import kagglehub

import os
import cv2
from PIL import Image

import pandas as pd
import seaborn as sns

import timeit
import matplotlib.pyplot as plt

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("deepak001/imagecolordetection")

print("Path to dataset files:", path)

100%|██████████| 68.6M/68.6M [00:03<00:00, 23.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1


In [3]:
imgs = []
for root, dirs, files in os.walk(path):
  print(root, files)
  if root == '/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/grey':
    print(root, files)
    for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                full_path = os.path.join(root, file)
                img_grey = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
                imgs.append((full_path, img_grey))
  elif root == '/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/color':
    print(root, files)
    for file in files:

            if file.endswith(('.jpg', '.jpeg', '.png')):
                full_path = os.path.join(root, file)
                print(os.path.isfile(full_path))
                img_c = cv2.imread(full_path)
                imgs.append((full_path, img_c))


/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1 []
/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Use_Case []
/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Use_Case/Normal Images ['_DS_Store']
/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Use_Case/Normal Images/color ['1001Q0VC99T_30_20180326T061833212Z.jpg', '1001LX28B8J_20_20180329T234750046Z.jpg', '1001DKR7X0Z_20_20180319T122901351Z.jpg', '1001XP8S294_20_20180405T101212553Z.jpg', '1004N380PLH_20_20180321T205807476Z.jpg']
/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Use_Case/Normal Images/grey ['10BK4BK61M7_20_20180123T141020265Z.jpg', '10BHWTD2G2D_20_20180316T173955683Z.jpg', '10BJC86SH4K_20_20180121T194651121Z.jpg', '10BHRCD56CJ_20_20180306T091656486Z.jpg', '10BHQXFBPX4_30_20180211T024028376Z.jpg']
/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Use_Case/Challenging Images ['_

In [4]:
for full_path, image in imgs:
  print(image)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 254]
  [255 255 254]]

 [[255 255 255]
  [188 188 188]
  [118 118 118]
  ...
  [119 119 119]
  [221 221 221]
  [255 255 254]]

 [[255 255 255]
  [118 118 118]
  [255 255 255]
  ...
  [183 183 183]
  [191 191 191]
  [255 255 255]]

 ...

 [[255 255 255]
  [119 119 119]
  [255 255 255]
  ..

In [9]:
def rgb_to_gray(r, g, b):    return 0.2989 * r + 0.5870 * g + 0.1140 * b

In [10]:
class Filter():
    """Класс Filter предоставляет методы для применения различных фильтров к растровым изображениям.
    """
    @staticmethod
    def sobel_filter(raster_map, _type):
        """Применяет фильтр Собеля к растровому изображению.

        Args:
            raster_map: Растровое изображение, к которому будет применен фильтр
            _type: Тип растрового изображения. Может быть "pgm" или "ppm".

        Returns:
            new_raster_map: Растровое изображение с примененным фильтром Собеля.
        """
        # Формируем ядра оператора Собеля
        matrix_x = [
            [1, 0, -1],
            [2, 0, -2],
            [1, 0, -1]
        ]
        matrix_y = [
            [1, 2, 1],
            [0, 0, 0],
            [-1, -2, -1]
        ]
        # Подгатавливаем новую мапу той же размерности
        new_raster_map = copy.copy(raster_map)
        # Формируем мапу на основе pgm
        if _type == "pgm":
            for y in range(1, len(raster_map) - 1):
                for x in range(1, len(raster_map[y]) - 1):
                    a = raster_map[(y - 1):(y + 2), (x - 1):(x + 2)]
                    G_x = np.sum(np.matmul(np.array(matrix_x), np.array(a)))
                    G_y = np.sum(np.matmul(np.array(matrix_y), np.array(a)))
                    new_raster_map[y][x] = np.sqrt(G_x ** 2 + G_y ** 2)
        # Формируем мапу на основе ppm
        if _type == "ppm":
            for y in range(1, len(raster_map) - 1):
                for x in range(1, len(raster_map[y]) - 1):
                    a = raster_map[(y - 1):(y + 2), (x - 1):(x + 2)]
                    for j in range(len(a)):
                        for i in range(len(a[j])):
                            b, g, r = a[j][i]
                            a[j][i] = rgb_to_gray(r, g, b)
                    G_x = np.sum(np.matmul(np.array(matrix_x), np.array(a)))
                    G_y = np.sum(np.matmul(np.array(matrix_y), np.array(a)))
                    c = np.sqrt(G_x ** 2 + G_y ** 2)
                    new_raster_map[y][x] = [c, c, c]
                if y % 10 == 0:
                    print(f'{y}/{len(raster_map)}')
        return new_raster_map

In [11]:
def apply_own_sobel(image, full_path):
    """ Применение самописного оператора Собеля с засеканием времени работы алгоритма.
    Args:
            image: Растровое изображение, к которому будет применен фильтр
            full_path: Путь до файла.

        Returns:
            new_raster_map: Растровое изображение с примененным фильтром Собеля.
            elapsed_time: Время работы алгоритма.
    """
    if full_path.startswith('/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/grey'):
        _type = "pgm"
        raster_map = image
    elif full_path.startswith('/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/color'):
        _type = "ppm"
        raster_map = image

    # Вызов метода sobel_filter класса Filter
    start_time = timeit.default_timer()
    new_raster_map = Filter.sobel_filter(raster_map, _type)
    elapsed_time = timeit.default_timer() - start_time
    # Сохранить изображение с примененным фильтром Собеля
    cv2.imwrite(full_path.replace('.jpg', '_sobel.jpg'), new_raster_map)
    return new_raster_map, elapsed_time

In [ ]:
sobel_results = {}
for full_path, image in imgs:
    print(f"Обрабатываю файл: {full_path}")
    new_raster_map, elapsed = apply_own_sobel(image, full_path)
    plt.figure(figsize=(20,10))
    plt.subplot(131),plt.imshow(image, cmap='gray')
    plt.title('Original'), plt.xticks([]), plt.yticks([])
    plt.subplot(132),plt.imshow(new_raster_map, cmap='gray')
    plt.title('Sobel'), plt.xticks([]), plt.yticks([])

    plt.show()
    full_path = full_path.replace("/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/grey/", "")
    full_path = full_path.replace("/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/color/", "")
    sobel_results[full_path] = elapsed

Обрабатываю файл: /root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/color/clr_test_11.jpg
10/1578
20/1578
30/1578
40/1578
50/1578
60/1578
70/1578
80/1578
90/1578
100/1578
110/1578
120/1578
130/1578
140/1578
150/1578
160/1578
170/1578
180/1578
190/1578
200/1578
210/1578
220/1578
230/1578
240/1578
250/1578
260/1578
270/1578
280/1578
290/1578
300/1578
310/1578
320/1578
330/1578
340/1578
350/1578
360/1578
370/1578
380/1578
390/1578
400/1578
410/1578
420/1578
430/1578
440/1578
450/1578
460/1578
470/1578
480/1578
490/1578
500/1578
510/1578
520/1578
530/1578
540/1578
550/1578
560/1578
570/1578
580/1578
590/1578
600/1578
610/1578
620/1578
630/1578
640/1578
650/1578
660/1578
670/1578
680/1578
690/1578
700/1578
710/1578
720/1578
730/1578
740/1578
750/1578
760/1578
770/1578
780/1578
790/1578
800/1578
810/1578
820/1578
830/1578
840/1578
850/1578
860/1578
870/1578
880/1578
890/1578
900/1578
910/1578
920/1578
930/1578
940/1578
950/1578
960/1578
970/1578
980/1578
9

In [ ]:
df = pd.DataFrame.from_dict(sobel_results, orient='index', columns=['Время'])
df.index.name = 'Имя файла'
df.reset_index(inplace=True)

sns.set(style="whitegrid")
ax = sns.barplot(x="Имя файла", y="Время", data=df)
ax.set_title('Время обработки изображений фильтром Собеля')
ax.set_xlabel('Имя файла')
ax.set_ylabel('Время (сек)')

plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# **Фильтр Собеля из opencv**

In [ ]:

import timeit
import matplotlib.pyplot as plt

def apply_sobel(image):
    start_time = timeit.default_timer()
    sobelx = cv2.Sobel(image, -1, 1, 0, ksize=5)
    sobely = cv2.Sobel(image, -1, 0, 1, ksize=5)
    elapsed_time = timeit.default_timer() - start_time
    return sobelx, sobely, elapsed_time

sobel_results = {}
for full_path, image in imgs:
    print(f"Обрабатываю файл: {full_path}")
    sobel_x, sobel_y, elapsed = apply_sobel(image)
    plt.figure(figsize=(20,10))
    plt.subplot(131),plt.imshow(image, cmap='gray')
    plt.title('Original'), plt.xticks([]), plt.yticks([])
    plt.subplot(132),plt.imshow(sobel_x, cmap='gray')
    plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
    plt.subplot(133),plt.imshow(sobel_y, cmap='gray')
    plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])

    plt.show()
    full_path = full_path.replace("/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/grey/", "")
    full_path = full_path.replace("/root/.cache/kagglehub/datasets/deepak001/imagecolordetection/versions/1/Dataset/Test/color/", "")
    sobel_results[full_path] = elapsed

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame.from_dict(sobel_results, orient='index', columns=['Время'])
df.index.name = 'Имя файла'
df.reset_index(inplace=True)

sns.set(style="whitegrid")
ax = sns.barplot(x="Имя файла", y="Время", data=df)
ax.set_title('Время обработки изображений фильтром Собеля из opencv')
ax.set_xlabel('Имя файла')
ax.set_ylabel('Время (сек)')

plt.xticks(rotation=90)
plt.tight_layout()
plt.show()
